In [54]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader 
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [55]:
data_df = pd.read_csv("riceClassification.csv")

In [56]:
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [57]:
data_df["Class"].value_counts()

Class
1    9985
0    8200
Name: count, dtype: int64

In [58]:
data_df.dropna(inplace=True)

In [59]:
data_df.drop(['id'],axis=1,inplace=True)

In [60]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [61]:
print(data_df.shape)

(18185, 11)


In [62]:
original_df = data_df.copy()

In [63]:
for column in data_df.columns:
    print(max(data_df[column]))
    data_df[column]=data_df[column]/max(data_df[column])

10210
183.2114344
82.55076212
0.9667736672
11008
114.0165591
0.8865730584
508.511
0.9047483132
3.911844673
1


In [64]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [65]:
X=np.array(data_df.iloc[:,:-1])
print(X.shape)

(18185, 10)


In [66]:
Y=np.array(data_df.iloc[:,-1:])
print(Y.shape)

(18185, 1)


In [67]:
X_train,x_test,Y_train,y_test=train_test_split(X,Y,train_size=0.7,random_state=42,shuffle=True)

In [68]:
print(X_train.shape,x_test.shape)
print(y_test.shape,Y_train.shape)

(12729, 10) (5456, 10)
(5456, 1) (12729, 1)


In [69]:
X_test,X_val,Y_test,Y_val=train_test_split(x_test,y_test,test_size=0.5,random_state=42,shuffle=True)

In [74]:
print(X_test.shape)
print(X_val.shape)
print(Y_test.shape)
print(Y_val.shape)

(2728, 10)
(2728, 10)
(2728, 1)
(2728, 1)


In [88]:
#pytorch data loaded function

class dataset(Dataset):
    def __init__(self,X,Y):
        self.X=torch.tensor(X,dtype=torch.float32).to(device)
        self.Y=torch.tensor(Y,dtype=torch.float32).to(device)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]
    
  

In [89]:
training_data = dataset(X_train,Y_train)
testing_data = dataset(X_test,Y_test)
validation_data = dataset(X_val,Y_val)

In [90]:
train_dataloader=DataLoader(training_data,batch_size=8,shuffle=True)
test_dataloader=DataLoader(testing_data,batch_size=8,shuffle=True)
val_dataloader=DataLoader(validation_data,batch_size=8,shuffle=True)

In [91]:
print(train_dataloader)

In [93]:
for x,y in train_dataloader:
    print(x)
    print("=====")
    print(y)
    break

tensor([[0.6696, 0.9235, 0.6325, 0.9839, 0.6356, 0.8183, 0.5600, 0.7285, 0.6920,
         0.8284],
        [0.6447, 0.9039, 0.6247, 0.9829, 0.6154, 0.8029, 0.5924, 0.7152, 0.6911,
         0.8209],
        [0.6792, 0.7457, 0.7958, 0.9069, 0.6435, 0.8242, 0.6652, 0.6565, 0.8643,
         0.5316],
        [0.5315, 0.6990, 0.6642, 0.9348, 0.5048, 0.7291, 0.6596, 0.5911, 0.8342,
         0.5971],
        [0.9688, 0.8804, 0.9586, 0.9013, 0.9293, 0.9843, 0.8349, 0.7811, 0.8707,
         0.5211],
        [0.7548, 0.7963, 0.8290, 0.9135, 0.7197, 0.8688, 0.6686, 0.6939, 0.8597,
         0.5450],
        [0.6747, 0.8848, 0.6734, 0.9716, 0.6535, 0.8214, 0.5602, 0.7343, 0.6863,
         0.7454],
        [0.9155, 0.8904, 0.8933, 0.9227, 0.8659, 0.9568, 0.6580, 0.7633, 0.8616,
         0.5655]])
=====
tensor([[1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.]])


In [94]:
HIDDEN_NEURONS=10
class MyModel(nn.Module):
    #layers
    def __init__(self):
        super(MyModel,self).__init__()
        self.input_layer= nn.Linear(X.shape[1], HIDDEN_NEURONS)
        self.linear=nn.Linear(HIDDEN_NEURONS,1)
        self.sigmoid=nn.Sigmoid()
    #how the data goes to the model
    def forward(self,x):
        x= self.input_layer(x)
        x=self.linear(x)
        x=self.sigmoid(x)
        return x
            

In [96]:
model=MyModel().to(device)

In [100]:
summary(model,(X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [101]:
criterion= nn.BCELoss()

In [103]:
optimizer=Adam(model.parameters(),lr=1e-3)

In [114]:
total_loss_train_plot = []
total_loss_val_plot = []
total_acc_train_plot = []
total_acc_val_plot = []

epochs = 10
for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0
    total_acc_val =0
    total_loss_val=0
    
    for data in train_dataloader:
        inputs,labels = data
        prediction = model(inputs)
       
        batch_loss= criterion(prediction,labels)
        total_loss_train+=batch_loss.item()
        
        acc = (prediction.round()==labels)
        print(acc)
        break
    break

    


tensor([[ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True]])
